In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
race_results_df = spark.read.format("delta").load(f"{incpresentation_folder_path}/race_results_dt") \
.filter(f"file_date = '{v_file_date}'") 

In [0]:
race_year_list = df_column_to_list(race_results_df, 'race_year')

In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.format("delta").load(f"{incpresentation_folder_path}/race_results_dt") \
.filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, when, count, col

constructor_standings_df = race_results_df \
.groupBy("race_year", "team") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1, True)).alias("wins"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
display(final_df)

race_year,team,total_points,wins,rank
2021,Mercedes,60.0,1,1
2021,Red Bull,53.0,1,2
2021,McLaren,41.0,0,3
2021,Ferrari,34.0,0,4
2021,AlphaTauri,8.0,0,5
2021,Aston Martin,5.0,0,6
2021,Alpine F1 Team,3.0,0,7
2021,Alfa Romeo,0.0,0,8
2021,Williams,0.0,0,8
2021,Haas F1 Team,0.0,0,8


In [0]:
merge_condition = "tgt.team = src.team AND tgt.race_year = src.race_year"
merge_delta_data(final_df, 'f1_inc_presentation', 'constructor_standings_dt', incpresentation_folder_path, merge_condition, 'race_year')
